<a href="https://colab.research.google.com/github/Jason-Durr/AIFinalProject/blob/master/HW/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jason Durr
# Task 1

For this task I chose to use data for the Microsoft Stock price including its open, previous day high, previous day low and previous day volume to predict if the closing price would be higher or lower than the opening price using 5 day intervals at a time.  This data is from 2011 to 2016 and came from https://www.kaggle.com/dgawlik/nyse?select=fundamentals.csv. The metric I used to evaluate this was accuracy and was comparing against a 50% chance of just guessing up or down.  For the RNN i used an 8 node recurrent layer plus a batch normalization layer which was recomended on the tensorflow RNN page followed by a 2 node dense output layer with a sigmoid activation function to get a probability of going up or down

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"jasondurr","key":"a288e7df4fa66f235cefea7adda21090"}'}

In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d dgawlik/nyse

mkdir: cannot create directory ‘/root/.kaggle’: File exists
100% 30.7M/30.7M [00:00<00:00, 51.5MB/s]



In [3]:
!unzip nyse.zip

Archive:  nyse.zip
  inflating: fundamentals.csv        
  inflating: prices-split-adjusted.csv  
  inflating: prices.csv              
  inflating: securities.csv          


In [153]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.model_selection import TimeSeriesSplit

In [14]:
data = pd.read_csv('prices-split-adjusted.csv')
data

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
...,...,...,...,...,...,...,...
851259,2016-12-30,ZBH,103.309998,103.199997,102.849998,103.930000,973800.0
851260,2016-12-30,ZION,43.070000,43.040001,42.689999,43.310001,1938100.0
851261,2016-12-30,ZTS,53.639999,53.529999,53.270000,53.740002,1701200.0
851262,2016-12-30,AIV,44.730000,45.450001,44.410000,45.590000,1380900.0


In [169]:
data2 = pd.DataFrame(data.loc[data['symbol'] == "MSFT"]).set_index("date")
data2['low'] = data2['low'].shift(periods=1)
data2['high'] = data2['high'].shift(periods=1)
data2['volume'] = data2['volume'].shift(periods=1)
df = data2.dropna().drop("symbol",axis=1)

tscv = TimeSeriesSplit(n_splits=2)

i = 0
for train_index, test_index in tscv.split(df):
  if i == 0:
    train = df.iloc[train_index]
    dev = df.iloc[test_index]
  else:
    test = df.iloc[dev_index]




def SplitData(df): 
    y = (df.close>df.open).astype(int)
    x = df.drop('close', axis = 1) 
    x = (x  - x.mean()) / x.std() 
    return x, y 

x_train, y_train = SplitData(train) 
x_dev, y_dev = SplitData(dev)
x_test, y_test = SplitData(test) 

In [201]:
input_dim = 4

model = keras.Sequential([
      layers.SimpleRNN(8,input_shape=(None,input_dim),activation="relu"),
      layers.BatchNormalization(),
      layers.Dense(2,"softmax")              
])


model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer="sgd",
    metrics=["accuracy"],
)


In [171]:
train_data_gen = TimeseriesGenerator(x_train.to_numpy(),y_train.to_numpy(),length=5,stride=1)
dev_data_gen = TimeseriesGenerator(x_dev.to_numpy(),y_dev.to_numpy(),length=5,stride=1)
test_data_gen = TimeseriesGenerator(x_test.to_numpy(),y_test.to_numpy(),length=5,stride=1)

In [207]:
model.fit(train_data_gen,validation_data=dev_data_gen, epochs=300
)

Epoch 1/300
10/10 [==============================] - 0s 11ms/step - loss: 0.6645 - accuracy: 0.5817 - val_loss: 0.7612 - val_accuracy: 0.4948
Epoch 2/300
10/10 [==============================] - 0s 7ms/step - loss: 0.6646 - accuracy: 0.5877 - val_loss: 0.7622 - val_accuracy: 0.5069
Epoch 3/300
10/10 [==============================] - 0s 10ms/step - loss: 0.6643 - accuracy: 0.5783 - val_loss: 0.7622 - val_accuracy: 0.5103
Epoch 4/300
10/10 [==============================] - 0s 9ms/step - loss: 0.6642 - accuracy: 0.5860 - val_loss: 0.7622 - val_accuracy: 0.5137
Epoch 5/300
10/10 [==============================] - 0s 8ms/step - loss: 0.6647 - accuracy: 0.5885 - val_loss: 0.7616 - val_accuracy: 0.4966
Epoch 6/300
10/10 [==============================] - 0s 8ms/step - loss: 0.6643 - accuracy: 0.5834 - val_loss: 0.7617 - val_accuracy: 0.5155
Epoch 7/300
10/10 [==============================] - 0s 8ms/step - loss: 0.6643 - accuracy: 0.5885 - val_loss: 0.7609 - val_accuracy: 0.4983
Epoch 8/300

In [208]:
model.evaluate(test_data_gen)

3/3 [==============================] - 0s 5ms/step - loss: 0.7989 - accuracy: 0.4841


[0.7989359498023987, 0.48414984345436096]

In [203]:
model2 = keras.Sequential([
      layers.LSTM(8,input_shape=(None,input_dim),activation="relu"),
      layers.BatchNormalization(),
      layers.Dense(2,activation="softmax")              
])


model2.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer="sgd",
    metrics=["accuracy"],
)

In [206]:
model2.fit(train_data_gen,validation_data=dev_data_gen, epochs=300
)

Epoch 1/300
10/10 [==============================] - 2s 36ms/step - loss: 1.0177 - accuracy: 0.4902 - val_loss: 0.7269 - val_accuracy: 0.5258
Epoch 2/300
10/10 [==============================] - 0s 9ms/step - loss: 0.8437 - accuracy: 0.4987 - val_loss: 0.7210 - val_accuracy: 0.5189
Epoch 3/300
10/10 [==============================] - 0s 9ms/step - loss: 0.7855 - accuracy: 0.5073 - val_loss: 0.7177 - val_accuracy: 0.5086
Epoch 4/300
10/10 [==============================] - 0s 9ms/step - loss: 0.7679 - accuracy: 0.5047 - val_loss: 0.7158 - val_accuracy: 0.5103
Epoch 5/300
10/10 [==============================] - 0s 9ms/step - loss: 0.7583 - accuracy: 0.5021 - val_loss: 0.7143 - val_accuracy: 0.5292
Epoch 6/300
10/10 [==============================] - 0s 9ms/step - loss: 0.7512 - accuracy: 0.5133 - val_loss: 0.7133 - val_accuracy: 0.5326
Epoch 7/300
10/10 [==============================] - 0s 9ms/step - loss: 0.7451 - accuracy: 0.5184 - val_loss: 0.7125 - val_accuracy: 0.5309
Epoch 8/300


## Part 2

This model does not have as much of an overfitting problem as the basic RNN does, this is most likely because having the additional information from the previous week makes it less likely to get lost in the noise of single day changes


In [209]:
model2.evaluate(test_data_gen)

3/3 [==============================] - 0s 5ms/step - loss: 0.7557 - accuracy: 0.5274


[0.7557085752487183, 0.5273775458335876]

# Task 2

For this task I chose to use Wor2vec pre-trained embedding with help from https://colab.research.google.com/github/practical-nlp/practical-nlp/blob/master/Ch3/05_Pre_Trained_Word_Embeddings.ipynb?authuser=1#scrollTo=n0LLCGJyjykp
and chose to use Heterogeneous Euclidean-Overlap Metric which I found in this paper: https://repub.eur.nl/pub/15911/ which is weighting the euclidian distance by the range of each value.  This makes sense because in regular euclidian distance some values will all be closer together than others so this is a way to even that out and get a more balanced distance measure.

In [215]:

!pip install scikit-learn==0.21.3
!pip install wget==3.2
!pip install gensim==3.6.0
!pip install psutil==5.4.8
!pip install spacy==2.2.4

     |████████████████████████████████| 6.7 MB 7.6 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=4f997b51a0e9a89aaa04bd44929b8ac3628c767edc34f55d98625201182faf3b
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [216]:
import os
import wget
import gzip
import shutil

gn_vec_path = "GoogleNews-vectors-negative300.bin"
if not os.path.exists("GoogleNews-vectors-negative300.bin"):
    if not os.path.exists("../Ch2/GoogleNews-vectors-negative300.bin"):
        #Downloading the reqired model
        if not os.path.exists("../Ch2/GoogleNews-vectors-negative300.bin.gz"):
            if not os.path.exists("GoogleNews-vectors-negative300.bin.gz"):
                wget.download("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz")
            gn_vec_zip_path = "GoogleNews-vectors-negative300.bin.gz"
        else:
            gn_vec_zip_path = "../Ch2/GoogleNews-vectors-negative300.bin.gz"
        #Extracting the required model
        with gzip.open(gn_vec_zip_path, 'rb') as f_in:
            with open(gn_vec_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    else:
        gn_vec_path = "../Ch2/" + gn_vec_path

print(f"Model at {gn_vec_path}")

Model at GoogleNews-vectors-negative300.bin


In [217]:
from gensim.models import Word2Vec, KeyedVectors
pretrainedpath = gn_vec_path

w2v_model = KeyedVectors.load_word2vec_format(pretrainedpath, binary=True)

In [306]:
mean = w2v_model.vectors.mean(axis=0)
std = w2v_model.vectors.std(axis=0)
max = w2v_model.vectors.max(axis=0)
min = w2v_model.vectors.min(axis=0)

In [307]:
from scipy import spatial

def dissimilarity(word1,word2):

  vec1 = w2v_model[word1] 
  vec2 = w2v_model[word2]

  return np.linalg.norm((vec1-vec2)/(max-min))


In [310]:
word1 = input("Enter word 1: ").strip().lower()
word2 = input("Enter word 2: ").strip().lower()
print("Cosine Similarity:", 1-spatial.distance.cosine(w2v_model[word1], w2v_model[word2]) )
print("Disimilarity:", dissimilarity(word1,word2) )

Enter word 1: test
Enter word 2: exam
Cosine Similarity: 0.5065851211547852
Disimilarity: 0.80642796
